In [20]:
import tkinter as tk
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
import matplotlib.pyplot as plt

# Variables globales para el canvas y el subplot
canvas = None
ax = None

piston_image_path = "Assets/asset-espejo.png"
piston_image_path2 = "Assets/asset-gris.jpg"

def get_image(path, zoom):
    img = plt.imread(path)
    return OffsetImage(img, zoom=zoom)

# Ruta de la imagen del pistón con diferentes tamaños
piston_image_zoom = 0.009
piston_image_zoom2 = 0.0055

def calcular_angulo_paralelo(Px, Py):
    return np.arctan2(Py, Px)

def calcular_angulo_perpendicular(Px, Py):
    return np.arctan2(Px, -Py)

def calcular_pistones():
    try:
        # Obtener parámetros de entrada
        L = float(entry_L.get())
        B = float(entry_B.get())
        D = float(entry_D.get())
        dmax = float(entry_dmax.get())
        Px = float(entry_Px.get())
        Py = float(entry_Py.get())

        # Verificar que los valores sean válidos
        if L <= 0 or B <= 0 or D <= 0 or dmax <= 0:
            raise ValueError("Todos los valores deben ser mayores que cero.")

        # Verificar restricciones
        if D > dmax:
            raise ValueError("La longitud mínima de los pistones debe ser menor que la máxima.")
        
        # Calcular el ángulo de orientación del espejo perpendicular al punto P
        theta = calcular_angulo_perpendicular(Px, Py)
        
        # Calcular las longitudes de los pistones
        x1 = D + (L / 2) * np.sin(theta)
        x2 = D - (L / 2) * np.sin(theta)

        # Mostrar resultados en la interfaz
        result_x1.set(f"x1 (Pistón 1): {x1:.2f}")
        result_x2.set(f"x2 (Pistón 2): {x2:.2f}")

        # Llamar a la función para graficar
        graficar(L, B, Px, Py, theta, x1, x2)
    except ValueError as e:
        messagebox.showerror("Error", str(e))
    except Exception as e:
        messagebox.showerror("Error", f"Error inesperado: {str(e)}")

def graficar(L, B, Px, Py, theta, x1, x2):
    global canvas, ax

    # Limpiar el contenido anterior del gráfico
    ax.clear()

    # Coordenadas de los extremos del espejo
    mirror_x = [-L / 2, L / 2]
    mirror_y = [x1, x2]

    # Coordenadas de la base
    base_x = [-B / 2, B / 2]
    base_y = [0, 0]

    # Coordenadas de los pistones dinámicos
    piston1_x = [base_x[0], mirror_x[0]]  # Conectar el extremo izquierdo de la base con el extremo izquierdo del espejo
    piston1_y = [base_y[0], mirror_y[0]]  # Coordenadas verticales dinámicas
    piston2_x = [base_x[1], mirror_x[1]]  # Conectar el extremo derecho de la base con el extremo derecho del espejo
    piston2_y = [base_y[1], mirror_y[1]]

    # Dibujar el espejo secundario
    ax.plot(mirror_x, mirror_y, label="Espejo Secundario", color="lightcyan", linewidth=5)
    ax.plot(mirror_x, mirror_y, color="azure", linewidth=2)

    # Dibujar la base del espejo
    ax.plot(base_x, base_y, linewidth=6, color="black", label="Base del Espejo")
    ax.plot(base_x, base_y, linewidth=1, linestyle="-.", color="crimson")

    # Dibujar la imagen del pistón en las posiciones inferiores de los pistones
    imagebox3 = AnnotationBbox(get_image(piston_image_path, piston_image_zoom), (-B / 2, 0), frameon=False)
    imagebox4 = AnnotationBbox(get_image(piston_image_path, piston_image_zoom), (B / 2, 0), frameon=False)
    ax.add_artist(imagebox3)
    ax.add_artist(imagebox4)

    # Dibujar los pistones inclinados
    ax.plot(piston1_x, piston1_y, linestyle="solid", linewidth=4, color="gray", label="Pistón 1")
    ax.plot(piston2_x, piston2_y, linestyle="solid", linewidth=4, color="gray", label="Pistón 2")
    ax.plot(piston1_x, piston1_y, linestyle="solid", linewidth=1, color="white")
    ax.plot(piston2_x, piston2_y, linestyle="solid", linewidth=1, color="white")

    # Dibujar la línea hacia el punto P
    ax.plot([0, Px], [0, Py], linestyle=":", color="crimson", label="Línea hacia el Punto P")
    ax.plot(Px, Py, 'o', color="crimson", label="Punto P")

    # Configuración de la gráfica
    ax.set_title("Telescopio de James Webb")
    ax.set_ylim(-30, 30)
    ax.set_xlim(-30, 30)
    ax.set_xlabel("Eje X")
    ax.set_ylabel("Eje Y")
    ax.axhline(0, color="black", linewidth=0.5, linestyle="--")
    ax.axvline(0, color="black", linewidth=0.5, linestyle="--")
    ax.grid(alpha=0.5)
    ax.legend()

    # Actualizar el canvas
    canvas.draw()

# Crear ventana principal
root = tk.Tk()
root.title("Datos para el Telescopio de James Webb")
root.configure(bg="dimgray")

# Variables de entrada
entry_L = tk.StringVar()
entry_B = tk.StringVar()
entry_D = tk.StringVar()
entry_dmax = tk.StringVar()
entry_Px = tk.StringVar()
entry_Py = tk.StringVar()

# Variables de salida
result_x1 = tk.StringVar()
result_x2 = tk.StringVar()

# Crear interfaz gráfica
tk.Label(root, text="Largo del Espejo (L):", bg="dimgray", fg="white").grid(row=0, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_L).grid(row=0, column=1, padx=5, pady=5)

tk.Label(root, text="Separación de Pistones (B):", bg="dimgray", fg="white").grid(row=1, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_B).grid(row=1, column=1, padx=5, pady=5)

tk.Label(root, text="Longitud Mínima de Pistones (D):", bg="dimgray", fg="white").grid(row=2, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_D).grid(row=2, column=1, padx=5, pady=5)

tk.Label(root, text="Longitud Máxima de Pistones (dmax):", bg="dimgray", fg="white").grid(row=3, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_dmax).grid(row=3, column=1, padx=5, pady=5)

tk.Label(root, text="Coordenada X del Punto P:", bg="dimgray", fg="white").grid(row=4, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_Px).grid(row=4, column=1, padx=5, pady=5)

tk.Label(root, text="Coordenada Y del Punto P:", bg="dimgray", fg="white").grid(row=5, column=0, padx=5, pady=5)
tk.Entry(root, textvariable=entry_Py).grid(row=5, column=1, padx=5, pady=5)

tk.Button(root, text="Calcular", command=calcular_pistones, bg="gray", fg="black").grid(row=6, column=0, columnspan=2, pady=10)

# Mostrar resultados
tk.Label(root, textvariable=result_x1, fg="aqua", bg="dimgray").grid(row=7, column=0, columnspan=2)
tk.Label(root, textvariable=result_x2, fg="violet", bg="dimgray").grid(row=8, column=0, columnspan=2)

# Frame para el gráfico
frame_graph = tk.Frame(root)
frame_graph.grid(row=9, column=0, columnspan=2, padx=10, pady=10)

# Crear figura inicial y canvas
fig = Figure(figsize=(6, 4), dpi=100)
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, frame_graph)
canvas.draw()
canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Iniciar la aplicación
root.mainloop()
